# Train HNN family (HNN, DGNet) for 'experiment-2body' problem

- Original HNN from hamiltonian_nn by Sam Greydanus

- Original DGNet form discrete-autograd by Takashi Matsubara

- modified and adapted by Jae Hoon (Daniel) Lee

In [1]:
import torch
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.float32)
import argparse
import numpy as np
import matplotlib.pyplot as plt

import os
import sys

EXPERIMENT_DIR = './experiment-2body'
sys.path.append(EXPERIMENT_DIR)

from data import get_dataset

In [2]:
def is_jupyter():
    return 'ipykernel' in sys.modules

In [3]:
def save_model_weights(model, directory, filename):
    try:
        if not filename.lower().endswith(('.pth', '.pt', '.tar')):
            filename += '.pth'
        
        if not os.path.exists(directory):
            os.makedirs(directory)

        full_path = os.path.join(directory, filename)
        torch.save(model.state_dict(), full_path)
        
        print(f"Model weights have successfully been saved: {full_path}")
    
    except Exception as e:
        print(f"Error occurred while saving the model: {e}")

In [4]:
def load_model_weights(model, filepath, device='cpu'):
    if not os.path.exists(filepath):
        print(f"Error: file not found: {filepath}")
        return

    try:
        state_dict = torch.load(filepath, map_location=torch.device(device))
        model.load_state_dict(state_dict)
        model.to(device)

        print(f"Model's weights have been successfully loaded: {filepath} (Device: {device})")
    
    except Exception as e:
        print(f"Error occurred while loading the model: {e}")

## DGNet

In [5]:
import dgnet

In [6]:
def get_dgnet_args():
    parser = argparse.ArgumentParser(description=None)
    parser.add_argument('--noretry', dest='noretry', action='store_true', help='not do a finished trial.')
    # network, experiments
    parser.add_argument('--input_dim', default=2 * 4, type=int, help='dimensionality of input tensor')
    parser.add_argument('--hidden_dim', default=200, type=int, help='hidden dimension of mlp')
    parser.add_argument('--learn_rate', default=1e-3, type=float, help='learning rate')
    parser.add_argument('--batch_size', default=200, type=int, help='batch_size')
    parser.add_argument('--nonlinearity', default='tanh', type=str, help='neural net nonlinearity')
    parser.add_argument('--total_steps', default=10000, type=int, help='number of gradient steps')
    parser.add_argument('--input_noise', default=0.0, type=int, help='std of noise added to inputs')
    # display
    parser.add_argument('--print_every', default=200, type=int, help='number of gradient steps between prints')
    parser.add_argument('--verbose', dest='verbose', action='store_true', help='verbose?')
    parser.add_argument('--name', default='2body', type=str, help='only one option right now')
    parser.add_argument('--seed', default=0, type=int, help='random seed')
    parser.add_argument('--save_dir', default=EXPERIMENT_DIR, type=str, help='where to save the trained model')
    # model
    parser.add_argument('--model', default='hnn', type=str, help='used model.')
    parser.add_argument('--solver', default='dg', type=str, help='used solver.')
    parser.add_argument('--friction', default=False, action="store_true", help='use friction parameter')
    parser.set_defaults(feature=True)

    if is_jupyter():
        return parser.parse_args([]) 
    else:
        return parser.parse_args()

In [7]:
def get_dgnet_model(device='cpu'):

    args = get_dgnet_args()
    model = dgnet.DGNet(args.input_dim, args.hidden_dim,
                        nonlinearity=args.nonlinearity, friction=args.friction, model=args.model, solver=args.solver)
    model = model.to(device)
    return model

In [8]:
def dgnet_train(args):
    ''' import from the current directory structure '''
    from hamiltonian_nn.utils import L2_loss

    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    dtype = torch.get_default_dtype()
    torch.set_grad_enabled(False)

    # set random seed
    torch.manual_seed(args.seed)
    np.random.seed(args.seed)

    # init model and optimizer
    model = dgnet.DGNet(args.input_dim, args.hidden_dim,
                        nonlinearity=args.nonlinearity, model=args.model, solver=args.solver)
    model = model.to(device)
    optim = torch.optim.Adam(model.parameters(), args.learn_rate, weight_decay=0)

    # arrange data
    t_span = 25
    length = 500
    dt = t_span / (length - 1)
    data = get_dataset(args.name, args.save_dir, verbose=True, timesteps=length, t_span=[0, t_span])
    train_x = torch.tensor(data['coords'], requires_grad=True, device=device, dtype=dtype)
    test_x = torch.tensor(data['test_coords'], requires_grad=True, device=device, dtype=dtype)
    train_dxdt = torch.tensor(data['dcoords'], device=device, dtype=dtype)
    test_dxdt = torch.tensor(data['test_dcoords'], device=device, dtype=dtype)
    x_reshaped = train_x.view(-1, length, args.input_dim)
    x1 = x_reshaped[:, :-1].contiguous().view(-1, args.input_dim)
    x2 = x_reshaped[:, 1:].contiguous().view(-1, args.input_dim)
    dxdt = ((x2 - x1) / dt).detach()

    # vanilla train loop
    stats = {'train_loss': [], 'test_loss': []}
    for step in range(args.total_steps + 1):
        with torch.enable_grad():
            # train step
            ixs = torch.randperm(x1.shape[0])[:args.batch_size]
            dxdt_hat = model.discrete_time_derivative(x1[ixs], dt=dt, x2=x2[ixs])
            dxdt_hat += args.input_noise * torch.randn_like(x1[ixs])  # add noise, maybe
            loss = L2_loss(dxdt[ixs], dxdt_hat)
        loss.backward()
        optim.step()
        optim.zero_grad()

        # run test data
        test_ixs = torch.randperm(test_x.shape[0], device=device)[:args.batch_size]
        test_dxdt_hat = model.time_derivative(test_x[test_ixs])
        test_dxdt_hat += args.input_noise * torch.randn_like(test_x[test_ixs])  # add noise, maybe
        test_loss = L2_loss(test_dxdt[test_ixs], test_dxdt_hat)

        # logging
        stats['train_loss'].append(loss.item())
        stats['test_loss'].append(test_loss.item())
        if args.verbose and step % args.print_every == 0:
            print("step {}, train_loss {:.4e}, test_loss {:.4e}"
                  .format(step, loss.item(), test_loss.item()))
            if args.friction:
                print("friction g =", model.g.detach().cpu().numpy())

    train_dxdt_hat = model.time_derivative(train_x)
    train_dist = (train_dxdt - train_dxdt_hat)**2
    test_dxdt_hat = model.time_derivative(test_x)
    test_dist = (test_dxdt - test_dxdt_hat)**2
    print('Final train loss {:.4e}\nFinal test loss {:.4e}'
          .format(train_dist.mean().item(), test_dist.mean().item()))
    stats['final_train_loss'] = train_dist.mean().item()
    stats['final_test_loss'] = test_dist.mean().item()

    return model

In [9]:
def train_dgnet_main():
    args = get_dgnet_args()

    # Explicilty set args.verbose attribute to True
    args.verbose = True
    args.print_every = 200

    # save
    os.makedirs(args.save_dir + '/results') if not os.path.exists(args.save_dir + '/results') else None
    label = ''
    label = label + '-{}-{}'.format(args.model, args.solver)
    label = label + '-friction' if args.friction else label
    label = label + '-seed{}'.format(args.seed)
    name = args.name
    result_path = '{}/results/dg-{}{}'.format(args.save_dir, name, label)
    path_tar = '{}.tar'.format(result_path)
    path_pkl = '{}.pkl'.format(result_path)
    path_txt = '{}.txt'.format(result_path)

    if os.path.exists(path_txt):
        if args.noretry:
            exit()
        else:
            os.remove(path_txt)

    model = dgnet_train(args)
    torch.save(model.state_dict(), path_tar)
        
    return model

In [10]:
dgnet_model = train_dgnet_main()

Successfully loaded data from ./experiment-2body/2body-orbits-dataset.pkl
step 0, train_loss 6.3007e-02, test_loss 5.0414e-02
step 200, train_loss 1.1355e-03, test_loss 8.8807e-04
step 400, train_loss 3.4787e-04, test_loss 2.7645e-04
step 600, train_loss 3.5661e-04, test_loss 2.1784e-04
step 800, train_loss 1.7680e-04, test_loss 1.8877e-04
step 1000, train_loss 1.2389e-04, test_loss 1.3109e-04
step 1200, train_loss 8.3940e-05, test_loss 7.5953e-05
step 1400, train_loss 4.0289e-05, test_loss 4.8356e-05
step 1600, train_loss 3.3955e-05, test_loss 3.5355e-05
step 1800, train_loss 4.0630e-05, test_loss 2.8098e-05
step 2000, train_loss 3.5671e-05, test_loss 2.6532e-05
step 2200, train_loss 2.7439e-05, test_loss 2.4724e-05
step 2400, train_loss 7.6412e-05, test_loss 2.2852e-05
step 2600, train_loss 1.7805e-05, test_loss 1.8627e-05
step 2800, train_loss 1.8993e-05, test_loss 2.8659e-05
step 3000, train_loss 1.9339e-05, test_loss 1.6054e-05
step 3200, train_loss 1.7628e-05, test_loss 2.3954e-0

In [11]:
save_dir = EXPERIMENT_DIR + "/weights"
save_model_weights(dgnet_model, save_dir,"dgnet_2body")

Model weights have successfully been saved: ./experiment-2body/weights/dgnet_2body.pth


## HNN 
(Hamiltonian Neural Netowork: modified version for GPU support: named as HNN2)

In [12]:
def get_hnn2_args():
    parser = argparse.ArgumentParser(description=None)
    parser.add_argument('--input_dim', default=2*4, type=int, help='dimensionality of input tensor')
    parser.add_argument('--hidden_dim', default=200, type=int, help='hidden dimension of mlp')
    parser.add_argument('--learn_rate', default=1e-3, type=float, help='learning rate')
    parser.add_argument('--batch_size', default=200, type=int, help='batch_size')
    parser.add_argument('--input_noise', default=0.0, type=int, help='std of noise added to inputs')
    parser.add_argument('--nonlinearity', default='tanh', type=str, help='neural net nonlinearity')
    parser.add_argument('--total_steps', default=10000, type=int, help='number of gradient steps')
    parser.add_argument('--print_every', default=200, type=int, help='number of gradient steps between prints')
    parser.add_argument('--name', default='2body', type=str, help='only one option right now')
    parser.add_argument('--baseline', dest='baseline', action='store_true', help='run baseline or experiment?')
    parser.add_argument('--verbose', dest='verbose', action='store_true', help='verbose?')
    parser.add_argument('--field_type', default='solenoidal', type=str, help='type of vector field to learn')
    parser.add_argument('--seed', default=0, type=int, help='random seed')
    parser.add_argument('--save_dir', default=EXPERIMENT_DIR, type=str, help='where to save the trained model')
    parser.set_defaults(feature=True)

    if is_jupyter():
        return parser.parse_args([]) 
    else:
        return parser.parse_args()

In [13]:
class HNN2(torch.nn.Module):
    '''Learn arbitrary vector fields that are sums of conservative and solenoidal fields'''
    def __init__(self, input_dim, differentiable_model, field_type='solenoidal',
                    baseline=False, assume_canonical_coords=True):
        super(HNN2, self).__init__()
        self.baseline = baseline
        self.differentiable_model = differentiable_model
        self.assume_canonical_coords = assume_canonical_coords
        self.field_type = field_type

        # --- modification 1 for GPU support: register_buffer is used ---
        # self.M = self.permutation_tensor(input_dim) # original HNN code by Sam Greydanus
        M_tensor = self.permutation_tensor(input_dim)
        self.register_buffer('M_buffer', M_tensor) # register M tensor as buffer
        
    def forward(self, x):
        # traditional forward pass
        if self.baseline:
            return self.differentiable_model(x)

        y = self.differentiable_model(x)
        assert y.dim() == 2 and y.shape[1] == 2, "Output tensor should have shape [batch_size, 2]"
        return y.split(1,1)

    '''def rk4_time_derivative(self, x, dt):
        # need to be modified to support GPU in rk4 or to use torchdiffeq
        return rk4(fun=self.time_derivative, y0=x, t=0, dt=dt)'''

    def time_derivative(self, x, t=None, separate_fields=False):
        '''NEURAL ODE-STLE VECTOR FIELD'''
        if self.baseline:
            return self.differentiable_model(x)

        '''NEURAL HAMILTONIAN-STLE VECTOR FIELD'''
        F1, F2 = self.forward(x) # traditional forward pass

        # --- modification 2 for GPU support: use x's device when creating tensors ---
        conservative_field = torch.zeros_like(x, device=x.device) # device=x.device
        solenoidal_field = torch.zeros_like(x, device=x.device) # device=x.device

        if self.field_type != 'solenoidal':
            dF1 = torch.autograd.grad(F1.sum(), x, create_graph=True)[0] # gradients for conservative field
            # create eye on the same device where self.M_buffer lives
            eye = torch.eye(*self.M_buffer.shape, device=self.M_buffer.device)
            conservative_field = dF1 @ eye

        if self.field_type != 'conservative':
            dF2 = torch.autograd.grad(F2.sum(), x, create_graph=True)[0] # gradients for solenoidal field
            # self.M_buffer is used
            solenoidal_field = dF2 @ self.M_buffer.t()

        if separate_fields:
            return [conservative_field, solenoidal_field]

        return conservative_field + solenoidal_field

    def permutation_tensor(self, n):
        # returns a torch tensor and processed at __init__
        M = None
        if self.assume_canonical_coords:
            M = torch.eye(n)
            M = torch.cat([M[n//2:], -M[:n//2]])
        else:
            M = torch.ones(n,n)
            M *= 1 - torch.eye(n)
            M[::2] *= -1
            M[:,::2] *= -1
            for i in range(n):
                for j in range(i+1, n):
                    M[i,j] *= -1
        return M

In [14]:
def get_hnn2_model(device):
    from hamiltonian_nn.nn_models import MLP
    args = get_hnn2_args()
    output_dim = 2
    nn_model = MLP(args.input_dim, args.hidden_dim, output_dim, args.nonlinearity)
    # The modified HNN class above (HNN2: GPU version) is used.
    model = HNN2(args.input_dim, differentiable_model=nn_model,
              field_type=args.field_type) 
    model.to(device) 
    return model

In [15]:
def hnn2_train():
  # adapted to the current hnn-family directory sturctures
  from hamiltonian_nn.utils import L2_loss

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  torch.set_grad_enabled(True)
  
  args = get_hnn2_args()
  args.verbose = True

  # set random seed
  torch.manual_seed(args.seed)
  np.random.seed(args.seed)
  
  model = get_hnn2_model(device)

  optim = torch.optim.Adam(model.parameters(), args.learn_rate, weight_decay=0)

  # arrange data
  data = get_dataset(args.name, args.save_dir, verbose=True)

   # torch.Tensor() in the original HNN by Sam Greydanus, 
   # has been replaced by torch.tensor() with device here for GPU.
  x = torch.tensor( data['coords'], requires_grad=True, dtype=torch.float32, device=device)
  test_x = torch.tensor( data['test_coords'], requires_grad=True, dtype=torch.float32, device=device)
  dxdt = torch.tensor(data['dcoords'], dtype=torch.float32, device=device) 
  test_dxdt = torch.tensor(data['test_dcoords'], dtype=torch.float32, device=device) 

  # vanilla train loop
  stats = {'train_loss': [], 'test_loss': []}
  for step in range(args.total_steps+1):

    # train step
    ixs = torch.randperm(x.shape[0])[:args.batch_size] # exists in the original version of HNN 
    ixs_gpu = ixs.to(device) # Now, we should assign it to GPU.
    dxdt_hat = model.time_derivative(x[ixs_gpu]) # calculated in the GPU context, now.
    noise = args.input_noise * torch.randn(*x[ixs].shape) # add noise, maybe
    dxdt_hat += noise.to(device) # This also should be modified to be casted into GPU.
    loss = L2_loss(dxdt[ixs_gpu], dxdt_hat)
    loss.backward()
    grad = torch.cat([p.grad.flatten() for p in model.parameters()]).clone()
    optim.step() ; optim.zero_grad()

    # run test data
    test_ixs = torch.randperm(test_x.shape[0])[:args.batch_size]
    test_ixs_gpu = test_ixs.to(device) # assigned to GPU
    test_dxdt_hat = model.time_derivative(test_x[test_ixs_gpu]) # calculated in the GPU context, now.
    noise = args.input_noise * torch.randn(*test_x[test_ixs].shape) # add noise, maybe
    test_dxdt_hat += noise.to(device) # This also should be modified to be casted into GPU.
    test_loss = L2_loss(test_dxdt[test_ixs_gpu], test_dxdt_hat)

    # logging
    stats['train_loss'].append(loss.item())
    stats['test_loss'].append(test_loss.item())
    if args.verbose and step % args.print_every == 0:
      print("step {}, train_loss {:.4e}, test_loss {:.4e}, grad norm {:.4e}, grad std {:.4e}"
          .format(step, loss.item(), test_loss.item(), grad@grad, grad.std()))

  train_dxdt_hat = model.time_derivative(x)
  train_dist = (dxdt - train_dxdt_hat)**2
  test_dxdt_hat = model.time_derivative(test_x)
  test_dist = (test_dxdt - test_dxdt_hat)**2
  print('Final train loss {:.4e} +/- {:.4e}\nFinal test loss {:.4e} +/- {:.4e}'
    .format(train_dist.mean().item(), train_dist.std().item()/np.sqrt(train_dist.shape[0]),
            test_dist.mean().item(), test_dist.std().item()/np.sqrt(test_dist.shape[0])))
  return model, stats

In [16]:
hnn2_model, stats=hnn2_train()

Successfully loaded data from ./experiment-2body/2body-orbits-dataset.pkl
step 0, train_loss 5.6959e-02, test_loss 5.0372e-02, grad norm 9.6848e-03, grad std 4.7793e-04
step 200, train_loss 4.7525e-04, test_loss 3.9000e-04, grad norm 6.9597e-05, grad std 4.0514e-05
step 400, train_loss 2.3650e-04, test_loss 1.6555e-04, grad norm 5.8871e-05, grad std 3.7262e-05
step 600, train_loss 2.4006e-04, test_loss 9.9555e-05, grad norm 1.7016e-05, grad std 2.0032e-05
step 800, train_loss 6.8406e-05, test_loss 6.3686e-05, grad norm 3.1591e-05, grad std 2.7296e-05
step 1000, train_loss 5.4871e-05, test_loss 4.5237e-05, grad norm 2.8903e-05, grad std 2.6109e-05
step 1200, train_loss 4.4448e-05, test_loss 3.4072e-05, grad norm 3.5755e-05, grad std 2.9040e-05
step 1400, train_loss 2.4692e-05, test_loss 4.0050e-05, grad norm 6.5014e-06, grad std 1.2383e-05
step 1600, train_loss 3.0835e-05, test_loss 1.9492e-05, grad norm 2.5659e-05, grad std 2.4600e-05
step 1800, train_loss 2.4295e-05, test_loss 3.1739e

In [17]:
save_dir = EXPERIMENT_DIR + "/weights"
save_model_weights(hnn2_model, save_dir,"hnn2_2body")

Model weights have successfully been saved: ./experiment-2body/weights/hnn2_2body.pth
